# History Compaction Demo

This notebook loads a sample chat history and uses the HistoryTools to compact it.

In [1]:
from core.config import LLMConfig
from core.llm import LLMClient
from tools.history import HistoryTools
from pathlib import Path
import json

config = LLMConfig.load()
llm = LLMClient(config)
history_tools = HistoryTools(lambda: llm)
SAMPLE = Path('tests/history_sample.json')
messages = json.loads(SAMPLE.read_text())
print('Loaded messages:', len(messages))


Loaded messages: 10


In [2]:
print(messages)

[{'role': 'system', 'content': 'You are a coding agent. Follow constraints strictly and prefer tool calls over verbose replies.'}, {'role': 'user', 'content': 'Create a polished, modern personal website that feels fast, friendly, and memorable, composed of six thoughtfully crafted sections that guide a visitor from first impression to action. Begin with a clean, persistent header that anchors the experience and provides effortless navigation: include a compact logo or nameplate on the left, a well-spaced horizontal menu on larger screens and a tidy menu button on small screens, and a clear visual indicator for the current page section. Make the header responsive and sticky so it remains visible while scrolling; add subtle hover and focus states for all interactive elements, generous hit targets for touch, and refined micro-interactions (for example, a soft underline that animates from the center outward). Ensure the header supports high-contrast themes, keyboard navigation, and proper 

In [3]:
# Tune compaction parameters for the demo
keep_last_n = 1  # reduce to force more of the history into the summarized head

result = history_tools.summarize_history(
    messages,
    keep_last_n=keep_last_n,
)
status = result.get('status')
compacted = result.get('messages', [])
print('Status:', status)
print('Before:', len(messages), 'After:', len(compacted))



Status: success
Before: 10 After: 10


In [4]:
from pprint import pprint
for idx, m in enumerate(compacted):
    print(f'--- message {idx} ({m.get("role")}) ---')
    print(m.get('content','')[:2000])


--- message 0 (system) ---
You are a coding agent. Follow constraints strictly and prefer tool calls over verbose replies.
--- message 1 (user) ---
Create a polished, modern personal website with six sections: header (sticky, responsive, dark/light mode toggle), hero (identity/value, CTA, visual), projects (grid, cards, filters), about (timeline, values, skills), contact (form, validation, privacy), footer (links, bio, legal). Use accessible, performant, and responsive design. No SVGs; use placeholders or TODOs for images.
--- message 2 (assistant) ---
Plan: create index.html, styles.css, App.jsx, components. Will scaffold structure and then fill content.
--- message 3 (user) ---
Add dark mode that follows prefers-color-scheme and basic animations.
--- message 4 (assistant) ---
Acknowledged. Using CSS prefers-color-scheme. Minimal animations on hover/focus.
--- message 5 (user) ---
Ensure no SVGs are generated. Use placeholders or TODOs when images are needed.
--- message 6 (assistant)

In [6]:
# Optional: write compacted history to a file for inspection
OUT = Path('tests/history_compacted.json')
OUT.write_text(json.dumps(compacted, indent=2))
print('Wrote', OUT)


Wrote tests/history_compacted.json
